In [1]:
import sys

from pathlib import Path
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mse

sys.path.append("/home/aseliverstov/projects/brain_signals/experiments/src")
from src.dataset import EEG
from src.features import calculate_summary_features
from src.augmentations import augment_timeseries

2025-02-02 02:42:47.703704: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-02 02:42:47.814020: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-02 02:42:47.931377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738453368.025492  256998 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738453368.062584  256998 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-02 02:42:48.305830: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/dataset")
intencities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
n_channels = 31

mona_dataset = EEG(data_dir=data_dir, intencities=intencities, n_channels=n_channels, image_name="Figs for spectra")
mona_dataset_val = EEG(data_dir=data_dir, intencities=intencities, n_channels=n_channels, image_name="Figs for spectra")

mona_dataset.load_to_dataframe(users=[1, 2, 3, 4, 5], signals=["theta", "alpha", "beta", "gamma"])
mona_dataset_val.load_to_dataframe(users=[1, 2, 3, 4, 5], signals=["delta"])

# Model

In [6]:
X_train, y_train = mona_dataset.X, mona_dataset.y
X_test, y_test = mona_dataset_val.X, mona_dataset_val.y

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = calculate_summary_features(X_train, n_windows=50)
X_test = calculate_summary_features(X_test, n_windows=50)

X_train.shape

(6200, 300)

In [7]:
model = Sequential()
model.add(Dense(16, input_shape= (X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=mse,
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=10, batch_size=16, verbose=1)

Epoch 1/10


/home/aseliverstov/.cache/pypoetry/virtualenvs/brain-signals-_5HxkjSc-py3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


388/388 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2759 - val_loss: 0.0825
Epoch 2/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0827 - val_loss: 0.0825
Epoch 3/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0834 - val_loss: 0.0826
Epoch 4/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0822 - val_loss: 0.0830
Epoch 5/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0836 - val_loss: 0.0828
Epoch 6/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0835 - val_loss: 0.0825
Epoch 7/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0821 - val_loss: 0.0832
Epoch 8/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0822 - val_loss: 0.0827
Epoch 9/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0842 - val_loss: 0.0836
Epoch 10/10
388/388 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0932 - val_loss: 0.0825


In [8]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 16)             │         4,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,917 (62.18 KB)

 Trainable params: 5,305 (20.72 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 10,612 (41.46 KB)